In [59]:
import os
import subprocess

In [93]:
gpu = 'ga_gpu_output.txt'
cpu = 'ga_cpu_output.txt'
tsp_dir = os.getcwd() + os.sep + 'TSPLIB'

In [98]:
tsp_files_list = os.listdir(tsp_dir)

In [56]:
def execGA(device, filename):
    if(device == 'gpu'):
        return subprocess.run()
    elif(device == 'cpu'):
    #running make GA_gpu
    compProc = subprocess.run(["./ga_gpu", filename], capture_output=True)
#     print(compProc)
    return compProc
    
    #running GA_gpu
#     process = subprocess.Popen("")

In [57]:
compProc = callGpuCode("gr17.tsp")

In [58]:
output = (compProcMake.stdout).decode("utf-8")
print(output)
idx = output.find("Execution time (GPU): ")
print(idx)
print(output[idx+22::])

-------------- 0 --------------
Solution: 2245.000000
-------------- 1 --------------
Solution: 2090.000000
-------------- 2 --------------
Solution: 2090.000000
-------------- 3 --------------
Solution: 2090.000000
-------------- 4 --------------
Solution: 2090.000000
-------------- 5 --------------
Solution: 2090.000000
-------------- 6 --------------
Solution: 2090.000000
-------------- 7 --------------
Solution: 2090.000000
-------------- 8 --------------
Solution: 2090.000000
-------------- 9 --------------
Solution: 2090.000000
-------------- 10 --------------
Solution: 2090.000000

GA converged in 11th generation with best Solution: 2090

Execution time (GPU): 1.80271 seconds

Number of cities: 	17
Population size: 	10000
Number of mutations: 	50
Max no of generations: 	10000

654
1.80271 seconds

Number of cities: 	17
Population size: 	10000
Number of mutations: 	50
Max no of generations: 	10000



In [81]:
def parseOutputFile(filename):
    fo = open(filename, 'r')
    lines = fo.readlines()
    num_gen = int(lines[0])
    res = {}
    res['g'] = num_gen
    solutions = []
    for i in range(num_gen):
        solutions.append(float(lines[i+1]))
    res['solns'] = solutions
    conv = int(lines[num_gen+1])
    res['converged'] = conv
    time = float(lines[num_gen+2])
    res['time'] = time
    return res

In [82]:
parseOutputFile(gpu)

{'g': 7,
 'solns': [2293.0, 2090.0, 2085.0, 2085.0, 2085.0, 2085.0, 2085.0],
 'converged': 1,
 'time': 1.1882}

['ulysses16.tsp', 'att532.tsp', 'pr226.tsp', 'kroA200.tsp', 'u1060.tsp', 'rd100.tsp', 'pla7397.tsp', 'dsj1000.tsp', 'd493.tsp', 'd2103.tsp', 'ts225.tsp', 'pr144.tsp', 'p654.tsp', 'pr152.tsp', 'd15112.tsp', 'swiss42.tsp', 'pla33810.tsp', 'brg180.tsp', 'eil51.tsp', 'lin318.tsp', 'u724.tsp', 'gr120.tsp', 'rat99.tsp', 'xray.problems', 'pr299.tsp', 'si1032.tsp', 'brd14051.tsp', 'd18512.tsp', 'gr96.tsp', 'kroC100.tsp', 'gr48.tsp', 'kroA100.tsp', 'pa561.tsp', 'dantzig42.tsp', 'fl3795.tsp', 'kroB200.tsp', 'fnl4461.tsp', 'gr24.tsp', 'rat195.tsp', 'solutions', 'kroB100.tsp', 'a280.tsp', 'vm1084.tsp', 'gr666.tsp', 'pr2392.tsp', 'brazil58.tsp', 'gr431.tsp', 'gil262.tsp', 'bayg29.tsp', 'rat575.tsp', 'lin105.tsp', 'rl1304.tsp', 'pr136.tsp', 'rl1889.tsp', 'fl1577.tsp', 'bier127.tsp', 'u159.tsp', 'rat783.tsp', 'rl5934.tsp', 'kroD100.tsp', 'fl1400.tsp', 'u1817.tsp', 'kroE100.tsp', 'nrw1379.tsp', 'u2319.tsp', 'pcb1173.tsp', 'pr76.tsp', 'd1655.tsp', 'vm1748.tsp', 'README.md', 'ch130.tsp', 'd198.tsp', 'al